#  Assignment 3: Structural Econometrics
## Eric Schulman

Solutions to ECO 388E assignment 3 at the University of Texas written by Eric Schulman

In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from statsmodels.base.model import GenericLikelihoodModel

### Part 1

The dynamic program for the firm maximizing future profits is expressed below:

$$V(a_t,i_t,\epsilon_{0t},\epsilon_{1t}) = \pi(a_t, i_t, \epsilon_{1t}, \epsilon_{0t}) + max_{i_{t+1}} \beta E(V(a_{t+1},i_{t+1},\epsilon_{0t+1},\epsilon_{1t+1}) |a_t, i_t; \theta)$$

Because of the conditional independence assumption $\epsilon_{it}$ is not serially correlated.

### Part 2

In class, $x_t$ was continuous. Here $a_t$ is discrete. Additionally, we did not parameterize $c(x_t, \theta)$. Here $c(a_t,\theta) = \theta a_t$, As a result, $c(0,\theta) = 0$

### Part 3

The code below is designed to calculate the value function using forward recursion. We determine the initial value using a contraction mapping iterating forward until it converges.

The value function is a 5x2 array. The rows contain the values when the state is $a_t$. The columns contain the value based on $i_t$.

In [2]:
BETA = .9
GAMMA = .5772 #euler's constant

def value_helper(a, theta1, cost, v_init):
    """helper function for calculating value function.
    
    given the value for the first period, calculate forward 
    'a' periods"""
    if a <= 0: #initial period
        return  v_init  
    else:
        v_next = value_helper(a-1, theta1, cost, v_init)
        v_0 = a*theta1 + BETA*(GAMMA + np.log( np.exp(v_next[-1][0]) + np.exp(v_next[-1][1]) ))  
        v_1 = cost + BETA*(GAMMA + np.log(  np.exp(v_next[-1][0]) + np.exp(v_next[-1][1]) )) 
        
        return np.concatenate( ( v_next,[[v_0,v_1]]) )


v = value_helper(5, -1, -3, [[0,0]])

In [3]:
def value_function(a, theta1, cost, v_init, error, maxiter):
    """solve for the first period of the value function
    with the contraction mapping loop
    
    You can choose how far into the future it goes by setting max_periods"""
    
    #only need to iterate 1 periods into the future
    v = value_helper(1, theta1, cost, v_init)
    
    #stop iterating when the last two periods look the same
    while ( maxiter >= 0  
           and ( abs(  v[1,0] - v[0,0] )  > error
           or abs(  v[1,1] - v[0,1] )  > error) ):
        
        #recompute value function until convergence
        return value_function(a, theta1, cost, [v[1,:]], error, maxiter-1)  
    
    return value_helper(a, theta1, cost, [v[1,:]])[1:,:]


print value_function(5, -1, -3, [[0,0]],.001, 100)

[[ -3.65524822  -5.65524822]
 [ -4.65600819  -5.65600819]
 [ -6.38899185  -6.38899185]
 [ -8.60678021  -7.60678021]
 [-11.04468667  -9.04468667]]


### Part 4

* Below I solve the model when $\theta_1 = -1$ and $R = -3$. 

* When $a_t = 2$, we can see that $V(2,0) - V(2,1) = 1$ so the firm chooses not to replace the engine. If the value of $\epsilon_{0t} - \epsilon_{1t}$ exceeds 1, then the firm will choose to replace the engine in period 2.

* I calculate the probability of this difference below using the exterme value distribution.

* Below I also calulate the value function when $a_t = 4$, $\epsilon_{0t} = 1$ and $\epsilon_{1t} =-1.5$. It is still cheaper to replace this period than to wait until period 5 to replace.

In [4]:
v = value_function(5, -1, -3, [[0,0]] , .001 , 100)
print '1. Value Function:'
print pd.DataFrame(v)

#difference between e0 and e1
diff = v[1,0] - v[1,1]
print '\n2. V(2,0) - V(2,1) = %s'%diff

#probability of this different
print '\n3. Likelihood: %s'%( np.exp(-diff)/(1+np.exp(-diff)) )

#PDV a = 4, e0= 1, e1=-1.5
print '\n4. PDV: %s'%np.maximum(-3 + 1 + BETA*v[3,0], -4 + -1.5 +BETA*v[3,1] )

1. Value Function:
           0         1
0  -3.655248 -5.655248
1  -4.656008 -5.656008
2  -6.388992 -6.388992
3  -8.606780 -7.606780
4 -11.044687 -9.044687

2. V(2,0) - V(2,1) = 1.0

3. Likelihood: 0.2689414213699951

4. PDV: -9.74610218495787


### Part 5 - 6 

Below I calculate the value of $\theta_1$ and $R$ and standard errors. 

The likelihood of $i_t$ is conditional on $a_t$ because my decision to replace this period depends on how old the engine is. The expected future costs, which I base my decision on, depend on the current age of the engine.

In [5]:
#load data into memory for part 5
data = np.loadtxt("data.asc")

In [6]:
class Rust(GenericLikelihoodModel):
    """class for estimating the values of R and theta"""
    
    def nloglikeobs(self, params, v=False):
        
        theta1, R = params
        
        # Input our data into the model
        i = self.exog[:,0] #reshape
        a = self.endog.astype(int)
        
        #solve value function based on params
        v = value_function(5, theta1, R, [[0,0]] , .01 , 100)
        
        #interpolate using scipy (easier than indexing)
        v0 = interp1d(range(1,6), v[:,0],fill_value="extrapolate")
        v1 = interp1d(range(1,6), v[:,1],fill_value="extrapolate")
        
        diff = v1(a) - v0(a)
    
        #calculate likelihood of each obs
        pr0 = 1/(1+np.exp(diff))
        pr1 = np.exp(diff)/(1+np.exp(diff))

        likelihood = (1-i)*pr0 + i*pr1
        return -np.log(likelihood).sum()
    
    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        if start_params == None:
            start_params = [1,1]
        return super(Rust, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    
    
model = Rust(data[:,0],data[:,1])

result = model.fit()
print(result.summary(xname=['theta_1', 'R']))

Optimization terminated successfully.
         Current function value: 0.425320
         Iterations: 55
         Function evaluations: 104
                                 Rust Results                                 
Dep. Variable:                      y   Log-Likelihood:                -425.32
Model:                           Rust   AIC:                             852.6
Method:            Maximum Likelihood   BIC:                             857.5
Date:                Wed, 19 Dec 2018                                         
Time:                        18:56:00                                         
No. Observations:                1000                                         
Df Residuals:                     999                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

### Part 7

A) To accomodate $\theta_{1A}$ and $\theta_{2A}$ we would need to modify the dynamic program as follows

$$V(a_t,i_t,\epsilon_{jt}) = \pi(a_t, i_t, \epsilon_{it}) + max_{i_{t+1}} \beta E(V(a_{t+1},i_{t+1},\epsilon_{jt+1}) |a_t, \epsilon_{jt}, i_t; \theta_i)$$



B) We would still have unobserved heterogeneity. However we could calculate the value of $\epsilon_{it-1}$ and include it in the likelihood function

$ L_{it} = Prob(i_{tj} | a_t, \epsilon_{jt-1} )$

C) If machines differ, then we can use fixed effects instead of making $\epsilon_{it}$ a state variable.

$L_{it} =  Prob(i_{tj} | a_t, \alpha{j} )$. We could simulate the $\alpha_j$'s

D) One way to solve it would be simulating back to period 1

E) Most there is not enough information to identify $\lambda$ seperately from $\alpha$

### Part 8

Below I estimate the model using the Hotz-Miller algorithm

In [44]:
def hm_prob(a_obs, i_obs):
    """calculate state pr"""
    
    df = np.array([a_obs,i_obs]).transpose()
    df = pd.DataFrame(df, columns=('a','i'))
    pr_state = df.groupby('a')
    pr_state = pr_state.count()/data.shape[0]

    return  np.array(pr_state)
    
    
def hm_transitions(a_max):
    """calculate transitions, deterministic
    in this case"""
    
    trans1 = np.zeros((a_max,a_max))
    trans1[:,0] = np.ones(a_max)
    
    trans0 = np.vstack( (np.identity(a_max-1), np.zeros(a_max-1)))
    trans0 = np.hstack( ( np.zeros((a_max,1)), trans0 ))
    trans0[a_max-1][a_max-1] = 1
    
    return trans0,trans1
    
    
def hm_value(a_max, theta1, cost, a_obs, i_obs, pr_state):
    """calculate value function using hotz miller approach"""
    #set up matrices, transition is deterministic
    #pr_state = hm_prob(a_max, a_obs, i_obs)
    trans0,trans1 = hm_transitions(a_max)
            
    #calculate value function for all state
    denom = (np.identity(a_max) - BETA*trans0.dot(1-pr_state) - BETA*trans1.dot(pr_state))
    
    a = np.arange(1,a_max+1).reshape(a_max,1)
    numer = ( (1-pr_state)*(theta1*a  + GAMMA - np.log(1-pr_state)) + 
                 pr_state*(cost+ GAMMA - np.log(pr_state) ) )
        
    value = np.linalg.inv(denom).dot(numer)
    return value
    
    
def hm_state_pr(a_max, theta1, cost, a_obs, i_obs, pr_state):
    """calculate kappa using value function"""
    
    value = hm_value(a_max, theta1, cost, a_obs, i_obs, pr_state)
    trans0,trans1 = hm_transitions(a_max)
    a = np.arange(1,a_max+1).reshape(a_max,1)
    
    
    delta1 = np.exp(cost + BETA*trans1.dot(value))
    delta0 = np.exp(a*theta1 + BETA*trans0.dot(value))
    
    return delta1/(delta1+delta0)


print hm_state_pr(5, -1, -3, data[:,0], data[:,1], hm_prob(data[:,0], data[:,1]))

[[0.21002378]
 [0.60351455]
 [0.88742366]
 [0.97989181]
 [0.99250737]]


In [47]:
class HotzMiller(GenericLikelihoodModel):
    """class for estimating the values of R and theta"""
    
    def __init__(self, a_max, pr_state, *args, **kwargs):
        super(HotzMiller, self).__init__( *args, **kwargs)
        self.a_max = a_max
        self.pr_state = pr_state
        
        
    def nloglikeobs(self, params): 
        theta1, R = params
        
        # Input our data into the model
        i = self.exog[:,0] #reshape
        a = self.endog.astype(int)
        
        #set up hm state pr
        state_pr = hm_state_pr(self.a_max, theta1, R, a, i, self.pr_state).transpose()[0]
        state_pr = interp1d(range(1,self.a_max+1), state_pr,fill_value="extrapolate")
        
        log_likelihood = i*np.log(state_pr(a)) + (1-i)*np.log(1-state_pr(a))
        
        return -log_likelihood.sum()
    
    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        if start_params == None:
            start_params = [1,1]
        return super(HotzMiller, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)

#initialize state probabilities
pr_state = hm_prob(data[:,0],data[:,1])
model_hm = HotzMiller(5, pr_state, data[:,0],data[:,1])
result_hm = model_hm.fit()
print(result_hm.summary(xname=['theta_1', 'R']))

Optimization terminated successfully.
         Current function value: 0.422985
         Iterations: 55
         Function evaluations: 106
                              HotzMiller Results                              
Dep. Variable:                      y   Log-Likelihood:                -422.99
Model:                     HotzMiller   AIC:                             848.0
Method:            Maximum Likelihood   BIC:                             852.9
Date:                Wed, 19 Dec 2018                                         
Time:                        19:24:55                                         
No. Observations:                1000                                         
Df Residuals:                     999                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

In [64]:
class AM():
    """A class for doing the AM contraction mapping"""
    def __init__(self,a_max, data):
        self.i = data[:,1]
        self.a = data[:,0]
        self.a_max = a_max
        
        #initilize pr using state
        self.pr_state =  hm_prob(self.a, self.i)
        self.model = HotzMiller(self.a_max, self.pr_state, self.a, self.i)
        self.result = self.model.fit(disp=0)
        self.theta1, self.cost = self.result.params
        

        
    def iterate(self,numiter):
        i = 0
        
        while(i < numiter):
            print self.pr_state
            self.pr_state = hm_state_pr(self.a_max, self.theta1, self.cost,
                                        self.a, self.i, self.pr_state)
            print self.pr_state
            
            self.model = HotzMiller(self.a_max, self.pr_state, self.a, self.i)
            
            print hm_value(self.a_max, self.theta1, self.cost,
                                        self.a, self.i, self.pr_state)
            
            self.result = self.model.fit(disp=0)
            
            self.theta1, self.cost = self.result.params 
            
            i = i +1
    

model_am = AM(5,data)
print model_am.result.summary(xname=['theta_1', 'R'])
model_am.iterate(1)
print model_am.result.summary(xname=['theta_1', 'R'])

                              HotzMiller Results                              
Dep. Variable:                      y   Log-Likelihood:                -422.99
Model:                     HotzMiller   AIC:                             848.0
Method:            Maximum Likelihood   BIC:                             852.9
Date:                Wed, 19 Dec 2018                                         
Time:                        19:32:07                                         
No. Observations:                1000                                         
Df Residuals:                     999                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1       -0.8413      0.050    -16.858      0.000      -0.939      -0.743
R             -3.3812      0.219    -15.408      0.0